# 数据准备

In [1]:
import backtrader as bt
from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import os
import backtrader.feeds as btfeeds
import datetime
code = '600893'
pos = 'sh' if code.find('600') == 0 else 'sz'
datapath = os.path.sep.join([get_root_path(),'data',pos,code,'{}.txt'.format(code)])
print(datapath)

data = btfeeds.GenericCSVData(
    dataname=datapath,
    fromdate=datetime.datetime(2019, 6, 1),
    dtformat=('%Y-%m-%d'),
    datetime=0,
    open=1,
    high=2,
    close=3,
    low=4,
    volume=5,
    amount=6)
print(data)
# cerebro = bt.Cerebro()
# cerebro.adddata(data)
# print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# cerebro.run()
# print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# cerebro.plot(style='bar')

D:\PROJECT_TW\git\finance\data\sh\600893\600893.txt


In [1]:
from lib.utils.lmdb_dataset import get_dataset
import lib.model.dp_model as dp
import importlib
from root import get_root_path
import lib.utils.sdata as sd
import os
import backtrader.feeds as btfeeds
import datetime
importlib.reload(dp)
importlib.reload(sd)
code = '600893'

class PandasDataExt(btfeeds.PandasData):
    lines = ('amount','returnd')
    params = (('amount', 7),('returnd',8))
    

db_path = os.path.sep.join([get_root_path(),'lmdb'])
dataframe = get_dataset(db_path,code)
sdata = sd.StockData(code, datas=dataframe)
dataframe = sdata.combine_income(ndays=5,buy=2)
dataframe['Datatime'] = dataframe.index.values
print(dataframe.head(10))
data = PandasDataExt(dataname=dataframe,   
                     datetime=-1,
                     fromdate=datetime.datetime(2019, 6, 1),
                          open=0,
                          high=1,
                          close=2,
                          low=3,
                          volume=4,
                          amount=5,
                          returnd=6)


D:\PROJECT_TW\git\finance\lib\utils\sdata.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.datas['Flag'] = list(map(lambda x: 0 if x <= buy else 1, self.datas.INCOME))


            Open  High  Close   Low   Volume     Amount    RETURN    INCOME  \
2005-01-05  3.85  3.96   3.94  3.80   783758   304341.5  2.604167  1.522843   
2005-01-06  3.88  3.95   3.93  3.85   293276   114618.9 -0.253807  4.071247   
2005-01-07  3.95  4.04   3.98  3.93   843481   336057.2  1.272265  4.271357   
2005-01-10  3.98  4.03   4.00  3.94   244178    97395.2  0.502513  3.750000   
2005-01-11  4.00  4.05   4.00  3.97   387684   154978.7  0.000000  3.750000   
2005-01-12  4.00  4.00   3.99  3.90   441909   174203.6 -0.250000  4.010025   
2005-01-13  3.98  4.18   4.09  3.87  1796552   728164.2  2.506266  1.466993   
2005-01-14  4.10  4.26   4.15  4.10  3474046  1451579.6  1.466993 -2.650602   
2005-01-17  4.10  4.10   4.02  3.97   944957   379889.5 -3.132530  1.492537   
2005-01-18  3.99  4.05   4.04  3.96   825610   330563.3  0.497512  0.990099   

             OUTCOME  Flag   Datatime  
2005-01-05 -0.253807     0 2005-01-05  
2005-01-06  1.272265     1 2005-01-06  
2005-01-07

In [36]:
??btfeeds.GenericCSVData

# 策略模拟

## 基础 

In [3]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        print(type(self.dataclose))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()        
        
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)
cerebro.adddata(data)
# Set our desired cash start , 启动资金
cerebro.broker.setcash(10000.0)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
<class 'backtrader.linebuffer.LineBuffer'>
2019-06-03, Close, 23.00
2019-06-04, Close, 22.81
2019-06-05, Close, 22.52
2019-06-05, BUY CREATE, 22.52
2019-06-06, Close, 21.92
2019-06-06, BUY CREATE, 21.92
2019-06-10, Close, 22.16
2019-06-11, Close, 22.64
2019-06-12, Close, 22.37
2019-06-13, Close, 22.40
2019-06-14, Close, 22.21
2019-06-17, Close, 21.80
2019-06-17, BUY CREATE, 21.80
2019-06-18, Close, 21.74
2019-06-18, BUY CREATE, 21.74
2019-06-19, Close, 22.12
2019-06-20, Close, 22.43
2019-06-21, Close, 22.64
2019-06-24, Close, 22.60
2019-06-25, Close, 22.76
2019-06-26, Close, 22.94
2019-06-27, Close, 23.00
2019-06-28, Close, 22.71
2019-07-01, Close, 23.16
2019-07-02, Close, 23.16
2019-07-03, Close, 23.25
2019-07-04, Close, 23.96
2019-07-05, Close, 23.66
2019-07-22, Close, 22.50
2019-07-22, BUY CREATE, 22.50
2019-07-23, Close, 22.61
2019-07-24, Close, 23.08
2019-07-25, Close, 23.00
2019-07-26, Close, 23.45
2019-07-29, Close, 23.45
2019-07-30, Close, 23.

## 个性化

In [5]:
import backtrader as bt
import backtrader.indicators as btind
import math

class SimpleMovingAverage1(bt.Indicator):
    lines = ('sma',)
    params = (('period', 20),)

    def next(self):
        dt = self.data.datetime.date(0)
        print('%s, %s' % (dt.isoformat(), self.data.amount.get(size=5)))        
#         print('%s' % (self.data.close.get(size=5)))        
        datasum = math.fsum(self.data.get(size=self.p.period))
        self.lines.sma[0] = datasum / self.p.period
        
        
class MyStrategy(bt.Strategy):
    
    def __init__(self):
#         self.sma = btind.SimpleMovingAverage(period=15)
        self.sma = SimpleMovingAverage1()
        self.dataclose = self.datas[0].close
        
#         print(self.lines.open)
        
    def next(self):
        if self.sma > self.data.close:
            self.log('BUY CREATE, %.2f,  %.2f' % (self.dataclose[0], len(self.dataclose)))
            self.buy()
        elif self.sma < self.data.close:
            self.log('SELL CREATE, %.2f, %.2f' % (self.dataclose[0], len(self.dataclose)))
            self.sell()
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)
cerebro.adddata(data)

# Set our desired cash start , 启动资金
cerebro.broker.setcash(10000.0)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2005-01-04, array('d')
2005-01-05, array('d')
2005-01-06, array('d')
2005-01-07, array('d')
2005-01-10, array('d', [260749.5, 304341.5, 114618.9, 336057.2, 97395.2])
2005-01-11, array('d', [304341.5, 114618.9, 336057.2, 97395.2, 154978.7])
2005-01-12, array('d', [114618.9, 336057.2, 97395.2, 154978.7, 174203.6])
2005-01-13, array('d', [336057.2, 97395.2, 154978.7, 174203.6, 728164.2])
2005-01-14, array('d', [97395.2, 154978.7, 174203.6, 728164.2, 1451579.6])
2005-01-17, array('d', [154978.7, 174203.6, 728164.2, 1451579.6, 379889.5])
2005-01-18, array('d', [174203.6, 728164.2, 1451579.6, 379889.5, 330563.3])
2005-01-19, array('d', [728164.2, 1451579.6, 379889.5, 330563.3, 170856.7])
2005-01-20, array('d', [1451579.6, 379889.5, 330563.3, 170856.7, 246826.8])
2005-01-21, array('d', [379889.5, 330563.3, 170856.7, 246826.8, 268400.6])
2005-01-24, array('d', [330563.3, 170856.7, 246826.8, 268400.6, 697781.6])
2005-01-25, array('d', [170856.7, 246826.8, 2684

2005-11-25, array('d', [362821.1, 462659.0, 350212.6, 561269.3, 796571.1])
2005-11-28, array('d', [462659.0, 350212.6, 561269.3, 796571.1, 267072.6])
2005-11-29, array('d', [350212.6, 561269.3, 796571.1, 267072.6, 630329.9])
2005-11-30, array('d', [561269.3, 796571.1, 267072.6, 630329.9, 338536.9])
2005-12-01, array('d', [796571.1, 267072.6, 630329.9, 338536.9, 270783.1])
2005-12-02, array('d', [267072.6, 630329.9, 338536.9, 270783.1, 207647.0])
2005-12-05, array('d', [630329.9, 338536.9, 270783.1, 207647.0, 293684.6])
2005-12-06, array('d', [338536.9, 270783.1, 207647.0, 293684.6, 428031.7])
2005-12-07, array('d', [270783.1, 207647.0, 293684.6, 428031.7, 228018.7])
2005-12-08, array('d', [207647.0, 293684.6, 428031.7, 228018.7, 190118.7])
2005-12-09, array('d', [293684.6, 428031.7, 228018.7, 190118.7, 214517.6])
2005-12-12, array('d', [428031.7, 228018.7, 190118.7, 214517.6, 934484.8])
2005-12-13, array('d', [228018.7, 190118.7, 214517.6, 934484.8, 539360.1])
2005-12-14, array('d', [1

2006-10-13, array('d', [1348350.8, 1219845.4, 917787.3, 809171.7, 1266916.8])
2006-10-16, array('d', [1219845.4, 917787.3, 809171.7, 1266916.8, 2672727.2])
2006-10-17, array('d', [917787.3, 809171.7, 1266916.8, 2672727.2, 885290.2])
2006-10-18, array('d', [809171.7, 1266916.8, 2672727.2, 885290.2, 564000.0])
2006-10-19, array('d', [1266916.8, 2672727.2, 885290.2, 564000.0, 1141961.8])
2006-10-20, array('d', [2672727.2, 885290.2, 564000.0, 1141961.8, 1014732.3])
2006-10-23, array('d', [885290.2, 564000.0, 1141961.8, 1014732.3, 1626791.0])
2006-10-24, array('d', [564000.0, 1141961.8, 1014732.3, 1626791.0, 2325660.4])
2006-10-25, array('d', [1141961.8, 1014732.3, 1626791.0, 2325660.4, 1343111.0])
2006-10-26, array('d', [1014732.3, 1626791.0, 2325660.4, 1343111.0, 1117294.9])
2006-10-27, array('d', [1626791.0, 2325660.4, 1343111.0, 1117294.9, 1051241.0])
2006-10-30, array('d', [2325660.4, 1343111.0, 1117294.9, 1051241.0, 607415.9])
2006-10-31, array('d', [1343111.0, 1117294.9, 1051241.0, 6

2009-07-09, array('d', [4868610.4, 8813192.0, 6204271.2, 12623264.0, 20258467.2])
2009-07-10, array('d', [8813192.0, 6204271.2, 12623264.0, 20258467.2, 10504001.6])
2009-07-13, array('d', [6204271.2, 12623264.0, 20258467.2, 10504001.6, 10087024.0])
2009-07-14, array('d', [12623264.0, 20258467.2, 10504001.6, 10087024.0, 7026848.8])
2009-07-15, array('d', [20258467.2, 10504001.6, 10087024.0, 7026848.8, 14179316.8])
2009-07-16, array('d', [10504001.6, 10087024.0, 7026848.8, 14179316.8, 24175932.8])
2009-07-17, array('d', [10087024.0, 7026848.8, 14179316.8, 24175932.8, 13521921.6])
2009-07-20, array('d', [7026848.8, 14179316.8, 24175932.8, 13521921.6, 11233743.2])
2009-07-21, array('d', [14179316.8, 24175932.8, 13521921.6, 11233743.2, 10039668.0])
2009-07-22, array('d', [24175932.8, 13521921.6, 11233743.2, 10039668.0, 8042512.0])
2009-07-23, array('d', [13521921.6, 11233743.2, 10039668.0, 8042512.0, 13440236.8])
2009-07-24, array('d', [11233743.2, 10039668.0, 8042512.0, 13440236.8, 1522150

2010-05-07, array('d', [14786710.4, 5538074.4, 5117916.0, 7523340.8, 13525620.8])
2010-05-10, array('d', [5538074.4, 5117916.0, 7523340.8, 13525620.8, 4885300.8])
2010-05-11, array('d', [5117916.0, 7523340.8, 13525620.8, 4885300.8, 3232862.0])
2010-05-12, array('d', [7523340.8, 13525620.8, 4885300.8, 3232862.0, 5265410.8])
2010-05-13, array('d', [13525620.8, 4885300.8, 3232862.0, 5265410.8, 8216793.6])
2010-05-14, array('d', [4885300.8, 3232862.0, 5265410.8, 8216793.6, 3756540.8])
2010-05-17, array('d', [3232862.0, 5265410.8, 8216793.6, 3756540.8, 4349646.0])
2010-05-18, array('d', [5265410.8, 8216793.6, 3756540.8, 4349646.0, 6288655.2])
2010-05-19, array('d', [8216793.6, 3756540.8, 4349646.0, 6288655.2, 3725460.0])
2010-05-20, array('d', [3756540.8, 4349646.0, 6288655.2, 3725460.0, 5205816.0])
2010-05-21, array('d', [4349646.0, 6288655.2, 3725460.0, 5205816.0, 4553064.8])
2010-05-24, array('d', [6288655.2, 3725460.0, 5205816.0, 4553064.8, 8706176.0])
2010-05-25, array('d', [3725460.0,

...

2011-09-06, BUY CREATE, 15.43,  1126.00
2011-09-07, BUY CREATE, 15.79,  1127.00
2011-09-08, BUY CREATE, 15.81,  1128.00
2011-09-09, BUY CREATE, 15.62,  1129.00
2011-09-13, BUY CREATE, 15.25,  1130.00
2011-09-14, BUY CREATE, 15.58,  1131.00
2011-09-15, BUY CREATE, 15.84,  1132.00
2011-09-16, BUY CREATE, 15.93,  1133.00
2011-09-19, BUY CREATE, 15.31,  1134.00
2011-09-20, BUY CREATE, 15.89,  1135.00
2011-09-21, SELL CREATE, 16.34, 1136.00
2011-09-22, BUY CREATE, 15.63,  1137.00
2011-09-23, BUY CREATE, 15.63,  1138.00
2011-09-26, BUY CREATE, 15.28,  1139.00
2011-09-27, BUY CREATE, 15.37,  1140.00
2011-09-28, BUY CREATE, 14.96,  1141.00
2011-09-29, BUY CREATE, 14.22,  1142.00
2011-09-30, BUY CREATE, 13.88,  1143.00
2011-10-10, BUY CREATE, 13.73,  1144.00
2011-10-11, BUY CREATE, 13.70,  1145.00
2011-10-12, BUY CREATE, 14.10,  1146.00
2011-10-13, BUY CREATE, 14.19,  1147.00
2011-10-14, BUY CREATE, 13.78,  1148.00
2011-10-17, BUY CREATE, 14.28,  1149.00
2011-10-18, BUY CREATE, 14.07,  1150.00


2013-10-14, SELL CREATE, 17.42, 1534.00
2013-10-15, SELL CREATE, 17.19, 1535.00
2013-10-16, BUY CREATE, 16.92,  1536.00
2013-10-17, BUY CREATE, 16.73,  1537.00
2013-10-18, BUY CREATE, 17.01,  1538.00
2013-10-21, SELL CREATE, 17.35, 1539.00
2013-10-22, SELL CREATE, 17.76, 1540.00
2013-10-23, SELL CREATE, 17.30, 1541.00
2013-10-24, BUY CREATE, 16.89,  1542.00
2013-10-25, BUY CREATE, 16.47,  1543.00
2013-10-28, BUY CREATE, 16.81,  1544.00
2013-10-29, BUY CREATE, 16.38,  1545.00
2013-10-30, SELL CREATE, 17.06, 1546.00
2013-10-31, BUY CREATE, 16.78,  1547.00
2013-11-01, BUY CREATE, 16.91,  1548.00
2013-11-04, BUY CREATE, 16.59,  1549.00
2013-11-05, BUY CREATE, 16.53,  1550.00
2013-11-06, BUY CREATE, 16.50,  1551.00
2013-11-07, BUY CREATE, 16.39,  1552.00
2013-11-08, BUY CREATE, 16.15,  1553.00
2013-11-11, BUY CREATE, 15.87,  1554.00
2013-11-12, BUY CREATE, 16.04,  1555.00
2013-11-13, BUY CREATE, 15.97,  1556.00
2013-11-14, SELL CREATE, 17.03, 1557.00
2013-11-15, SELL CREATE, 17.58, 1558.00


2015-06-18, BUY CREATE, 58.43,  1942.00
2015-06-19, BUY CREATE, 60.16,  1943.00
2015-06-23, BUY CREATE, 66.18,  1944.00
2015-06-24, BUY CREATE, 67.80,  1945.00
2015-06-25, BUY CREATE, 64.44,  1946.00
2015-06-26, BUY CREATE, 58.00,  1947.00
2015-06-29, BUY CREATE, 52.20,  1948.00
2015-06-30, BUY CREATE, 53.15,  1949.00
2015-07-01, BUY CREATE, 47.84,  1950.00
2015-07-02, BUY CREATE, 43.06,  1951.00
2015-07-03, BUY CREATE, 38.75,  1952.00
2015-07-06, BUY CREATE, 34.88,  1953.00
2015-07-07, BUY CREATE, 31.39,  1954.00
2015-07-08, BUY CREATE, 28.34,  1955.00
2015-07-09, BUY CREATE, 31.17,  1956.00
2015-07-10, BUY CREATE, 34.29,  1957.00
2015-07-13, BUY CREATE, 37.72,  1958.00
2015-07-14, BUY CREATE, 41.49,  1959.00
2015-07-15, BUY CREATE, 43.14,  1960.00
2015-07-16, BUY CREATE, 46.18,  1961.00
2015-07-17, SELL CREATE, 50.80, 1962.00
2015-07-20, SELL CREATE, 55.88, 1963.00
2015-07-21, SELL CREATE, 54.60, 1964.00
2015-07-22, SELL CREATE, 60.06, 1965.00
2015-07-23, SELL CREATE, 61.87, 1966.00


2017-03-28, SELL CREATE, 33.90, 2350.00
2017-03-29, BUY CREATE, 33.69,  2351.00
2017-03-30, BUY CREATE, 33.38,  2352.00
2017-03-31, BUY CREATE, 33.67,  2353.00
2017-04-05, SELL CREATE, 33.89, 2354.00
2017-04-06, BUY CREATE, 33.78,  2355.00
2017-04-07, SELL CREATE, 34.14, 2356.00
2017-04-10, SELL CREATE, 34.12, 2357.00
2017-04-11, SELL CREATE, 37.53, 2358.00
2017-04-12, SELL CREATE, 37.92, 2359.00
2017-04-13, SELL CREATE, 37.33, 2360.00
2017-04-14, SELL CREATE, 36.83, 2361.00
2017-04-17, SELL CREATE, 35.80, 2362.00
2017-04-18, SELL CREATE, 35.49, 2363.00
2017-04-19, SELL CREATE, 35.62, 2364.00
2017-04-20, SELL CREATE, 35.41, 2365.00
2017-04-21, BUY CREATE, 34.97,  2366.00
2017-04-24, BUY CREATE, 34.81,  2367.00
2017-04-25, BUY CREATE, 35.04,  2368.00
2017-04-26, BUY CREATE, 34.38,  2369.00
2017-04-27, BUY CREATE, 34.20,  2370.00
2017-04-28, BUY CREATE, 34.35,  2371.00
2017-05-02, BUY CREATE, 33.69,  2372.00
2017-05-03, BUY CREATE, 34.07,  2373.00
2017-05-04, BUY CREATE, 33.54,  2374.00


2018-11-27, BUY CREATE, 23.22,  2758.00
2018-11-28, BUY CREATE, 23.40,  2759.00
2018-11-29, BUY CREATE, 23.21,  2760.00
2018-11-30, BUY CREATE, 23.48,  2761.00
2018-12-03, SELL CREATE, 23.95, 2762.00
2018-12-04, SELL CREATE, 23.90, 2763.00
2018-12-05, BUY CREATE, 23.60,  2764.00
2018-12-06, BUY CREATE, 23.65,  2765.00
2018-12-07, BUY CREATE, 23.60,  2766.00
2018-12-10, BUY CREATE, 22.98,  2767.00
2018-12-11, BUY CREATE, 22.95,  2768.00
2018-12-12, BUY CREATE, 22.92,  2769.00
2018-12-13, BUY CREATE, 23.24,  2770.00
2018-12-14, BUY CREATE, 22.80,  2771.00
2018-12-17, BUY CREATE, 23.01,  2772.00
2018-12-18, BUY CREATE, 22.85,  2773.00
2018-12-19, BUY CREATE, 22.42,  2774.00
2018-12-20, BUY CREATE, 22.15,  2775.00
2018-12-21, BUY CREATE, 21.99,  2776.00
2018-12-24, BUY CREATE, 22.00,  2777.00
2018-12-25, BUY CREATE, 21.73,  2778.00
2018-12-26, BUY CREATE, 21.59,  2779.00
2018-12-27, BUY CREATE, 21.49,  2780.00
2018-12-28, BUY CREATE, 21.72,  2781.00
2019-01-02, BUY CREATE, 21.50,  2782.00


array('d')

# 策略应用

## 时间策略 

In [16]:
import lib.strategy.base as sbase
import lib.model.base_model as b_model
import importlib
import backtrader as bt
from root import get_root_path
importlib.reload(sbase)
importlib.reload(b_model)
# cerebro = bt.Cerebro()
# 修改成该模型方式，策略取当天时间 可参看 https://www.backtrader.com/docu/cerebro/cheat-on-open/cheat-on-open/
cerebro = bt.Cerebro(cheat_on_open=True)
param = {'code':'600803','model_path':os.path.sep.join([get_root_path(),'save','600893','times.bif'])}
cerebro.addstrategy(sbase.TimeStrategy,**param)
cerebro.adddata(data)
cerebro.broker.setcash(10000.0)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot()

Starting Portfolio Value: 10000.00
600803 D:\PROJECT_TW\git\finance\save\600893\times.bif
2019-06-04, 1, test [0.47986944 0.52013056]
2019-06-05, 2, test [0.53079876 0.46920124]
2019-06-05 Buy cash 19.959999999999127 Executed at price 22.89, 436
2019-06-06, 3, test [0.48097882 0.51902118]
2019-06-06 Sell cash 9825.599999999999 Executed at price 22.49, 0
2019-06-10, 0, test [0.47390696 0.52609304]
2019-06-11, 1, test [0.47986944 0.52013056]
2019-06-12, 2, test [0.53079876 0.46920124]
2019-06-12 Buy cash 17.19999999999709 Executed at price 22.600000000000005, 434
2019-06-13, 3, test [0.48097882 0.51902118]
2019-06-13 Sell cash 9782.199999999997 Executed at price 22.5, 0
2019-06-14, 4, test [0.45520417 0.54479583]
2019-06-17, 0, test [0.47390696 0.52609304]
2019-06-18, 1, test [0.47986944 0.52013056]
2019-06-19, 2, test [0.53079876 0.46920124]
2019-06-19 Buy cash 14.19999999999709 Executed at price 22.0, 444
2019-06-20, 3, test [0.48097882 0.51902118]
2019-06-20 Sell cash 9839.91999999999

[[<Figure size 573.134x429.851 with 4 Axes>]]

## 深度学习LSTM策略

In [38]:
import lib.strategy.deep as dbase
import lib.model.dp_model as d_model
import backtrader as bt
import importlib
from root import get_root_path
importlib.reload(dbase)
importlib.reload(d_model)
cerebro = bt.Cerebro()
param = {'code':'600803','num_feature':7,'model_path':os.path.sep.join([get_root_path(),'save','600893','lstm.pkh'])}
cerebro.addstrategy(dbase.DeepStrategy,**param)
cerebro.broker.setcash(10000.0)
cerebro.adddata(data)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 10000.00
2019-06-24 : cash:10000.0 current buy price :22.6  current buy size :392  value :10000.0 
2019-06-25,notify order Submitted
2019-06-25,notify order Accepted
2019-06-25,notify order Completed
2019-06-25,BUY COMPLETE, 22.60, 392.00, 22.60, 0.00
2019-06-25 : cash 1140.7999999999993 current sell price :22.76  current sell size :392 value :10062.72 
2019-06-26,notify order Submitted
2019-06-26,notify order Accepted
2019-06-26,notify order Completed
2019-06-26,SELL COMPLETE, 22.61, 0.00, 0.00, 0.00
2019-06-26 : cash:10003.92 current buy price :22.94  current buy size :386  value :10003.92 
2019-06-27,notify order Submitted
2019-06-27,notify order Accepted
2019-06-27,notify order Completed
2019-06-27,BUY COMPLETE, 22.94, 386.00, 22.94, 0.00
2019-06-27 : cash 1149.08 current sell price :23.0  current sell size :386 value :10027.08 
2019-06-28,notify order Submitted
2019-06-28,notify order Accepted
2019-06-28,notify order Completed
2019-06-28,SELL COMPLETE, 23

2019-10-14,SELL COMPLETE, 22.19, 0.00, 0.00, 0.00
2019-10-14 : cash:10941.569999999998 current buy price :22.12  current buy size :444  value :10941.569999999998 
2019-10-15,notify order Submitted
2019-10-15,notify order Accepted
2019-10-15,notify order Completed
2019-10-15,BUY COMPLETE, 22.12, 444.00, 22.12, 0.00
2019-10-15 : cash 1120.2899999999972 current sell price :21.84  current sell size :444 value :10817.249999999996 
2019-10-16,notify order Submitted
2019-10-16,notify order Accepted
2019-10-16,notify order Completed
2019-10-16,SELL COMPLETE, 21.84, 0.00, 0.00, 0.00
2019-10-16 : cash:10817.249999999998 current buy price :21.94  current buy size :443  value :10817.249999999998 
2019-10-17,notify order Submitted
2019-10-17,notify order Accepted
2019-10-17,notify order Completed
2019-10-17,BUY COMPLETE, 21.93, 443.00, 21.93, 0.00
2019-10-17 : cash 1102.2599999999984 current sell price :21.8  current sell size :443 value :10759.659999999998 
2019-10-18,notify order Submitted
2019-1